<a href="https://www.kaggle.com/code/riyosha/co2-prediction?scriptVersionId=191670875" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Exploratory Data Analysis

In [ ]:
# this notebook has picked many ideas from ambrosm's notebook on this dataset
# https://www.kaggle.com/code/ambrosm/pss3e20-eda-which-makes-sense/notebook


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

In [ ]:
data_path = '/kaggle/input/playground-series-s3e20'
train_data = pd.read_csv(data_path+'/train.csv')
test_data = pd.read_csv(data_path+'/test.csv')
samplesubmission = pd.read_csv(data_path+'/sample_submission.csv')


In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
samplesubmission.head()

In [ ]:
train_data.shape, test_data.shape, samplesubmission.shape

In [ ]:
test_data.shape[0]/train_data.shape[0]

Statistical Description

In [ ]:
'''
We can see that - 
1. Time range of data is from 2019 to 2021, and weekly data is provided
2. Min and Max emmissions are 0 to 3167.768
'''
train_data.describe(include='all')

In [ ]:
# Let's check out the distribution of the data 
plt.figure(figsize=(5, 3))
sns.histplot(train_data['emission'], kde=True)
plt.title('Distribution of Emissions')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.ylim(0, 4000) 
plt.show()

print('Skew: ', train_data['emission'].skew()) # skewed to the right - we'll figure out how to handle this (if needed) while building our model

In [ ]:
plt.figure(figsize=(12, 2))
sns.boxplot(x=train_data['emission'])
plt.title('Box Plot')
plt.xlabel('Emission')
plt.show()

# most of the emissions are below 500

In [ ]:
#missing values

print('Rows with at least 1 missing value: ', train_data.isna().any(axis=1).sum())

# almost all rows have some missing value, so we can't just drop the missing values. 
# We'll handle missing values EDA, before we train our model

In [ ]:
# Let's find out how many geographical points we have

print(train_data.groupby(['latitude', 'longitude']).size().sort_values())
print(test_data.groupby(['latitude', 'longitude']).size().sort_values())

# there are 497 distinct geographical points, each of which have 159 data points in training set
# and 49 data points in the test set

In [ ]:
# we can analyse emissions according to these 497 points
point_mean_emissions = train_data.groupby(['latitude','longitude']).emission.mean().sort_values()
point_mean_emissions

# alot of these points have mean 0 emissions => their prediction should likely be 0

Geo Visualisation

In [ ]:
# Let's visualise these points on a map 

import geopandas as gpd # geosatial data visualisation
from shapely.geometry import Point # representing data as a 2d or 3d point
import folium # for interactive maps
import matplotlib.colors as mcolors


In [ ]:
# we can get a heat map of these points on the geographic map 

point_mean_emissions = point_mean_emissions.reset_index()
geometry = gpd.points_from_xy(point_mean_emissions.longitude, point_mean_emissions.latitude)

# Create point geometries
geometry = gpd.points_from_xy(point_mean_emissions.longitude, point_mean_emissions.latitude)
geo_df = gpd.GeoDataFrame(
    point_mean_emissions, geometry=geometry
)

train_coords_map = folium.Map(prefer_canvas=True)

# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]

norm = plt.Normalize(vmin=np.log1p(point_mean_emissions.emission.min()), vmax=np.log1p(point_mean_emissions.emission.max()))
cmap = plt.get_cmap('coolwarm') 

def get_color(emission):
    return mcolors.to_hex(cmap(norm(np.log1p(emission))))
i=0
# Iterate through list and add a marker for each emission point
for coordinates in geo_df_list: 
    emiss = point_mean_emissions[(point_mean_emissions['latitude']==coordinates[0]) & (point_mean_emissions['longitude']==coordinates[1])]
    emiss = emiss['emission'].iloc[0]
    color = get_color(emiss)
    # Place the markers 
    train_coords_map.add_child(
        folium.CircleMarker(
            location=coordinates,
            radius = 1,
            weight = 4,
            zoom =10,
            fill=True,
            popup= 
            "Coordinates: " + str([round(x, 2) for x in geo_df_list[i]]),
            color = color),
        )
    i = i + 1
train_coords_map.fit_bounds(train_coords_map.get_bounds())
train_coords_map

# looks like the higher emission points are more centrally located.
# also, longitude decides the emission level much more than a point's latitude

Timeseries EDA

In [ ]:
#now let's analyse this data as a timeseries

plt.figure(figsize=(4,2))
sns.countplot(x='year', data=train_data)
plt.title('Year Countplot')
plt.show()

# each year has the same number of datapoints

In [ ]:
plt.figure(figsize=(10,2))
sns.countplot(x='week_no', data=train_data)
plt.title('Week Countplot')
plt.show()

plt.figure(figsize=(10,2))
sns.countplot(x='week_no', data=train_data[train_data['year']==2019])
plt.title('Week Countplot (2019)')
plt.show()

plt.figure(figsize=(10,2))
sns.countplot(x='week_no', data=train_data[train_data['year']==2020])
plt.title('Week Countplot (2020)')
plt.show()

plt.figure(figsize=(10,2))
sns.countplot(x='week_no', data=train_data[train_data['year']==2021])
plt.title('Week Countplot (2021)')
plt.show()

#each week has the same amount of data points as well, and each year contains equal data for all weeks
# datapoints are evenly spread across the weeks and years

In [ ]:
# to analyse the entire data as a time series, we'll consider the sum of weekly emissions
train_data['date'] = pd.to_datetime(train_data['year'].astype(str) + train_data['week_no'].astype(str) + '0', format='%Y%W%w')
weekly_emissions = train_data.groupby(['date'])[['emission']].sum()
weekly_emissions.plot(figsize=(12,2))
plt.show()

In [ ]:
#let's look at its TSA decomposition

import statsmodels.api as sm

rcParams['figure.figsize'] = (15,6)
decomposition = sm.tsa.seasonal_decompose(weekly_emissions, model='additive',period=52)
fig = decomposition.plot()
plt.show()

# we see a decreasing trend in 2020, which can be attributed to COVID.
# Near 2020 end, the emissions start rising again

In [ ]:
#now let's plot the time series of different geo points together

plt.figure(figsize=(13,4))
for _,point in train_data[['latitude','longitude']].drop_duplicates().iterrows():
    ts = train_data[(train_data['latitude']==point.latitude)&(train_data['longitude']==point.longitude)].emission
    plt.plot(range(len(ts)),ts)

plt.title('Time series for every geographical point')
for week in [0, 53, 106, 159]:
    plt.axvline(week, color='k', linestyle='--')

plt.xlabel('Week')
plt.ylabel('Emissions')
plt.show()

In [ ]:
# let's look at each point's trend to see if there are any anomalies
plt.figure(figsize=(13,7))
for _,point in train_data[['latitude','longitude']].drop_duplicates().iterrows():
    ts = train_data[(train_data['latitude']==point.latitude)&(train_data['longitude']==point.longitude)].emission
    decomp = sm.tsa.seasonal_decompose(ts, model='additive',period=52)
    plt.plot(range(len(decomp.trend)),decomp.trend)

plt.title('Trend for every geographical point')
for week in [0, 53, 106, 159]:
    plt.axvline(week, color='k', linestyle='--')

plt.xlabel('Week')
plt.ylabel('Emissions')
plt.show()

# all series dip at least a little in 2020 and then rise again
# the lowest emitting geo points don't seem to be affected much

Feature Analysis

In [ ]:
train_data.columns

In [ ]:
# we can pick the most correlated features to train our model on
top20_corrs = abs(train_data.drop(columns=['ID_LAT_LON_YEAR_WEEK']).corr()['emission']).sort_values(ascending = False).head(21)
top20_corrs
# as observed on the map earlier, latitude doesn't seem to be very correlated to a point's emission

In [ ]:
corr_matrix=train_data[list(top20_corrs.keys())].corr()
plt.figure(figsize=(15,10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', vmin=-1, vmax=1, linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

# we can see that some variables are highly correlated. 
# for starters, we can pick 3 features to train our model on (in addition to year and week_no)
# the first should intuitively be longitude already
# UvAerosolLayerHeight_aerosol_height is not correlated to longitude, so let this be the 2nd feature
# UvAerosolLayerHeight_aerosol_pressure is almost perfectly negatively correlated to vAerosolLayerHeight_aerosol_height, so we don't get any new info and skip this feature
# 